In [ ]:
####### 1. TRAINING (ONEHIGH) DATASET #####

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from mapie.classification import MapieClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasClassifier  # Scikit-Learn compatible wrapper

#  Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

#  **Load and Normalize OneHigh Dataset**
data_one_high = np.load("OneHigh_.npy", allow_pickle=True)

scaler = MinMaxScaler()
def normalize_data(data):
    for i in range(52):  # 52 NFC tag classes
        data_abs = np.abs(data[:, :, i])
        data[:, :, i] = scaler.fit_transform(data_abs)
    return data

OneHigh = normalize_data(data_one_high)

# **Updated CNN-ResNet-LSTM Model Architecture**
def build_model(input_shape, num_classes, dropout_rate=0.5):  # Increased dropout for better generalization
    input_layer = tf.keras.layers.Input(input_shape)

    # **Initial Convolutional Layer**
    x = tf.keras.layers.Conv1D(32, 
                               kernel_size=8, 
                               padding='same', 
                               activation='relu',
                               kernel_regularizer=regularizers.l2(0.001),
                               kernel_initializer='he_normal')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)

    # **Residual Block Function**
    def residual_block(x, filters, kernel_size=8):
        shortcut = x
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation='relu',
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation=None,
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = tf.keras.layers.Conv1D(filters, 
                                              kernel_size=1, 
                                              padding='same',
                                              kernel_regularizer=regularizers.l2(0.001),
                                              kernel_initializer='he_normal')(shortcut)
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    # **Residual Block and LSTM Layers**
    x = residual_block(x, 32, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # **LSTM Layer with GlorotUniform Initialization**
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(64, 
                             return_sequences=True,
                             kernel_initializer='glorot_uniform')(x)

    x = residual_block(x, 64, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # **Output Layer with Softmax Activation**
    outputs = tf.keras.layers.Dense(num_classes, 
                                    activation='softmax',
                                    kernel_regularizer=regularizers.l2(0.001),
                                    kernel_initializer='he_normal')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Reduced learning rate
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'],
                  weighted_metrics=['accuracy'])  
    return model

#  **Prepare OneHigh Dataset**
num_classes = 52
random_state = 42
calibration_size = 0.1  
test_size = 0.2  
train_size = 1 - (calibration_size + test_size)  

combination_name = "OneHigh"
dataset = OneHigh

#  **Prepare dataset**
X_combined = np.empty((dataset.shape[0] * dataset.shape[2], dataset.shape[1]))
labels = np.empty((dataset.shape[0] * dataset.shape[2]), dtype=int)  

for card in range(dataset.shape[2]):
    responses = dataset[:, :, card]
    start_idx = card * dataset.shape[0]
    end_idx = start_idx + dataset.shape[0]
    X_combined[start_idx:end_idx, :] = responses
    labels[start_idx:end_idx] = card  

#  **Split Data**
X_train, X_temp, y_train, y_temp = train_test_split(X_combined, labels, test_size=(test_size + calibration_size), stratify=labels, random_state=random_state)
X_test, X_calib, y_test, y_calib = train_test_split(X_temp, y_temp, test_size=(calibration_size / (test_size + calibration_size)), stratify=y_temp, random_state=random_state)

#  **Save Split Data**
os.makedirs('./Data_Splits', exist_ok=True)
np.save(f'./Data_Splits/x_train_{combination_name}.npy', X_train)
np.save(f'./Data_Splits/y_train_{combination_name}.npy', y_train)
np.save(f'./Data_Splits/x_test_{combination_name}.npy', X_test)
np.save(f'./Data_Splits/y_test_{combination_name}.npy', y_test)
np.save(f'./Data_Splits/x_calib_{combination_name}.npy', X_calib)
np.save(f'./Data_Splits/y_calib_{combination_name}.npy', y_calib)

#  **Set up Checkpoints & Early Stopping**
checkpoint_dir = "./tf_ckpts_OneHigh"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.weights.h5")  #  Fixed the format

#  Save the best model checkpoint based on validation accuracy
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,  
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

#  Stop training early if validation loss doesn't improve for 5 epochs
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

#  **Wrap Model in KerasClassifier for Scikit-Learn Compatibility**
model = KerasClassifier(
    model=build_model,
    input_shape=(X_train.shape[1], 1),
    num_classes=num_classes,
    dropout_rate=0.5,  # Updated dropout rate
    epochs=100,  
    batch_size=32,
    verbose=1,
    callbacks=[checkpoint_callback, early_stopping_callback]  # Include callbacks
)

#  **Train the model**
model.fit(X_train, y_train, validation_split=0.10)

#  **Save the Best Model Separately**
best_model_path = "MD/OneHigh_best.h5"
os.makedirs("MD", exist_ok=True)
model.model_.save(best_model_path)
print(f" Best model saved as {best_model_path}")

#  **Wrap and Fit MAPIE Classifier**
print(" Fitting MAPIE classifier...")
mapie_clf = MapieClassifier(estimator=model, method="score", cv="prefit")
mapie_clf.fit(X_calib, y_calib)
print(" MAPIE classifier fitted successfully.")

In [1]:
####### 2. TRAINING (ONELOW) DATASET #####

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from mapie.classification import MapieClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasClassifier  # Scikit-Learn compatible wrapper


#  **Set random seed for reproducibility**
tf.random.set_seed(42)
np.random.seed(42)

#  **Load and Normalize OneLow Dataset**
data_one_low = np.load("OneLow_.npy", allow_pickle=True)

scaler = MinMaxScaler()
def normalize_data(data):
    for i in range(52):  # 52 NFC tag classes
        data_abs = np.abs(data[:, :, i])
        data[:, :, i] = scaler.fit_transform(data_abs)
    return data

OneLow = normalize_data(data_one_low)

#  **Updated CNN-ResNet-LSTM Model Architecture**
def build_model(input_shape, num_classes, dropout_rate=0.5):
    input_layer = tf.keras.layers.Input(input_shape)

    # **Initial Convolutional Layer**
    x = tf.keras.layers.Conv1D(32, 
                               kernel_size=8, 
                               padding='same', 
                               activation='relu',
                               kernel_regularizer=regularizers.l2(0.001),
                               kernel_initializer='he_normal')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)

    # **Residual Block Function**
    def residual_block(x, filters, kernel_size=8):
        shortcut = x
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation='relu',
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation=None,
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = tf.keras.layers.Conv1D(filters, 
                                              kernel_size=1, 
                                              padding='same',
                                              kernel_regularizer=regularizers.l2(0.001),
                                              kernel_initializer='he_normal')(shortcut)
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    # **Residual Block and LSTM Layers**
    x = residual_block(x, 32, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # **LSTM Layer**
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(64, 
                             return_sequences=True,
                             kernel_initializer='glorot_uniform')(x)

    x = residual_block(x, 64, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # **Output Layer**
    outputs = tf.keras.layers.Dense(num_classes, 
                                    activation='softmax',
                                    kernel_regularizer=regularizers.l2(0.001),
                                    kernel_initializer='he_normal')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'],
                  weighted_metrics=['accuracy'])  
    return model

# **Prepare OneLow Dataset**
num_classes = 52
random_state = 42
calibration_size = 0.1  
test_size = 0.2  
train_size = 1 - (calibration_size + test_size)  

combination_name = "OneLow"
dataset = OneLow

# **Prepare dataset**
X_combined = np.empty((dataset.shape[0] * dataset.shape[2], dataset.shape[1]))
labels = np.empty((dataset.shape[0] * dataset.shape[2]), dtype=int)  

for card in range(dataset.shape[2]):
    responses = dataset[:, :, card]
    start_idx = card * dataset.shape[0]
    end_idx = start_idx + dataset.shape[0]
    X_combined[start_idx:end_idx, :] = responses
    labels[start_idx:end_idx] = card  

# **Split Data**
X_train, X_temp, y_train, y_temp = train_test_split(X_combined, labels, test_size=(test_size + calibration_size), stratify=labels, random_state=random_state)
X_test, X_calib, y_test, y_calib = train_test_split(X_temp, y_temp, test_size=(calibration_size / (test_size + calibration_size)), stratify=y_temp, random_state=random_state)

# **Save Split Data**
os.makedirs('./Data_Splits', exist_ok=True)
np.save(f'./Data_Splits/x_train_{combination_name}.npy', X_train)
np.save(f'./Data_Splits/y_train_{combination_name}.npy', y_train)
np.save(f'./Data_Splits/x_test_{combination_name}.npy', X_test)
np.save(f'./Data_Splits/y_test_{combination_name}.npy', y_test)
np.save(f'./Data_Splits/x_calib_{combination_name}.npy', X_calib)
np.save(f'./Data_Splits/y_calib_{combination_name}.npy', y_calib)

#  **Set up Checkpoints & Early Stopping**
checkpoint_dir = "./tf_ckpts_OneLow"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.weights.h5")  # Fixed the format

#  Save the best model checkpoint based on validation accuracy
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,  
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

#  Stop training early if validation loss doesn't improve for 5 epochs
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

model = KerasClassifier(
        model=build_model,
        input_shape=(X_train.shape[1], 1),
        num_classes=num_classes,
        dropout_rate=0.5,  
        epochs=100,  
        batch_size=32,
        verbose=1,
        callbacks=[checkpoint_callback, early_stopping_callback]  
    )

#  **Train the Model**
model.fit(X_train, y_train, validation_split=0.10)

#  **Save the Best Model Separately**
best_model_path = "MD/OneLow_best.h5"
os.makedirs("MD", exist_ok=True)
model.model_.save(best_model_path)
print(f" Best model saved as {best_model_path}")

# **Wrap and Fit MAPIE Classifier**
print(" Fitting MAPIE classifier...")
mapie_clf = MapieClassifier(estimator=model, method="score", cv="prefit")
mapie_clf.fit(X_calib, y_calib)
print(" MAPIE classifier fitted successfully.")

In [2]:
####### 3. TRAINING (TWOHIGH) DATASET #####

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from mapie.classification import MapieClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasClassifier  # Scikit-Learn compatible wrapper


# **Set random seed for reproducibility**
tf.random.set_seed(42)
np.random.seed(42)

# **Load and Normalize TwoHigh Dataset**
data_two_high = np.load("TwoHigh_.npy", allow_pickle=True)

scaler = MinMaxScaler()
def normalize_data(data):
    for i in range(52):  # 52 NFC tag classes
        data_abs = np.abs(data[:, :, i])
        data[:, :, i] = scaler.fit_transform(data_abs)
    return data

TwoHigh = normalize_data(data_two_high)

#  **Updated CNN-ResNet-LSTM Model Architecture**
def build_model(input_shape, num_classes, dropout_rate=0.5):
    input_layer = tf.keras.layers.Input(input_shape)

    # **Initial Convolutional Layer**
    x = tf.keras.layers.Conv1D(32, 
                               kernel_size=8, 
                               padding='same', 
                               activation='relu',
                               kernel_regularizer=regularizers.l2(0.001),
                               kernel_initializer='he_normal')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)

    # **Residual Block Function**
    def residual_block(x, filters, kernel_size=8):
        shortcut = x
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation='relu',
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation=None,
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = tf.keras.layers.Conv1D(filters, 
                                              kernel_size=1, 
                                              padding='same',
                                              kernel_regularizer=regularizers.l2(0.001),
                                              kernel_initializer='he_normal')(shortcut)
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    # **Residual Block and LSTM Layers**
    x = residual_block(x, 32, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # **LSTM Layer**
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(64, 
                             return_sequences=True,
                             kernel_initializer='glorot_uniform')(x)

    x = residual_block(x, 64, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # **Output Layer**
    outputs = tf.keras.layers.Dense(num_classes, 
                                    activation='softmax',
                                    kernel_regularizer=regularizers.l2(0.001),
                                    kernel_initializer='he_normal')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'],
                  weighted_metrics=['accuracy'])  
    return model

# **Prepare TwoHigh Dataset**
num_classes = 52
random_state = 42
calibration_size = 0.1  
test_size = 0.2  
train_size = 1 - (calibration_size + test_size)  

combination_name = "TwoHigh"
dataset = TwoHigh

# **Prepare dataset**
X_combined = np.empty((dataset.shape[0] * dataset.shape[2], dataset.shape[1]))
labels = np.empty((dataset.shape[0] * dataset.shape[2]), dtype=int)  

for card in range(dataset.shape[2]):
    responses = dataset[:, :, card]
    start_idx = card * dataset.shape[0]
    end_idx = start_idx + dataset.shape[0]
    X_combined[start_idx:end_idx, :] = responses
    labels[start_idx:end_idx] = card  

# **Split Data**
X_train, X_temp, y_train, y_temp = train_test_split(X_combined, labels, test_size=(test_size + calibration_size), stratify=labels, random_state=random_state)
X_test, X_calib, y_test, y_calib = train_test_split(X_temp, y_temp, test_size=(calibration_size / (test_size + calibration_size)), stratify=y_temp, random_state=random_state)

#  **Save Split Data**
os.makedirs('./Data_Splits', exist_ok=True)
np.save(f'./Data_Splits/x_train_{combination_name}.npy', X_train)
np.save(f'./Data_Splits/y_train_{combination_name}.npy', y_train)
np.save(f'./Data_Splits/x_test_{combination_name}.npy', X_test)
np.save(f'./Data_Splits/y_test_{combination_name}.npy', y_test)
np.save(f'./Data_Splits/x_calib_{combination_name}.npy', X_calib)
np.save(f'./Data_Splits/y_calib_{combination_name}.npy', y_calib)

# **Set up Checkpoints & Early Stopping**
checkpoint_dir = "./tf_ckpts_TwoHigh"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.weights.h5")  # Fixed the format

#  Save the best model checkpoint based on validation accuracy
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,  
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

#  Stop training early if validation loss doesn't improve for 5 epochs
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

model = KerasClassifier(
        model=build_model,
        input_shape=(X_train.shape[1], 1),
        num_classes=num_classes,
        dropout_rate=0.5,  
        epochs=100,  
        batch_size=32,
        verbose=1,
        callbacks=[checkpoint_callback, early_stopping_callback]  
    )

#  **Train the Model**
model.fit(X_train, y_train, validation_split=0.10)

# **Save the Best Model Separately**
best_model_path = "MD/TwoHigh_best.h5"
os.makedirs("MD", exist_ok=True)
model.model_.save(best_model_path)
print(f" Best model saved as {best_model_path}")

#  **Wrap and Fit MAPIE Classifier**
print(" Fitting MAPIE classifier...")
mapie_clf = MapieClassifier(estimator=model, method="score", cv="prefit")
mapie_clf.fit(X_calib, y_calib)
print(" MAPIE classifier fitted successfully.")

In [3]:
####### 4. TRAINING (TWOLOW) DATASET #####

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from mapie.classification import MapieClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasClassifier  # Scikit-Learn compatible wrapper


# **Set random seed for reproducibility**
tf.random.set_seed(42)
np.random.seed(42)

# **Load and Normalize TwoHigh Dataset**
data_two_Low = np.load("TwoLow_.npy", allow_pickle=True)

scaler = MinMaxScaler()
def normalize_data(data):
    for i in range(52):  # 52 NFC tag classes
        data_abs = np.abs(data[:, :, i])
        data[:, :, i] = scaler.fit_transform(data_abs)
    return data

TwoLow = normalize_data(data_two_Low)

#  **Updated CNN-ResNet-LSTM Model Architecture**
def build_model(input_shape, num_classes, dropout_rate=0.5):
    input_layer = tf.keras.layers.Input(input_shape)

    # **Initial Convolutional Layer**
    x = tf.keras.layers.Conv1D(32, 
                               kernel_size=8, 
                               padding='same', 
                               activation='relu',
                               kernel_regularizer=regularizers.l2(0.001),
                               kernel_initializer='he_normal')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)

    # **Residual Block Function**
    def residual_block(x, filters, kernel_size=8):
        shortcut = x
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation='relu',
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation=None,
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = tf.keras.layers.Conv1D(filters, 
                                              kernel_size=1, 
                                              padding='same',
                                              kernel_regularizer=regularizers.l2(0.001),
                                              kernel_initializer='he_normal')(shortcut)
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    # **Residual Block and LSTM Layers**
    x = residual_block(x, 32, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # **LSTM Layer**
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(64, 
                             return_sequences=True,
                             kernel_initializer='glorot_uniform')(x)

    x = residual_block(x, 64, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # **Output Layer**
    outputs = tf.keras.layers.Dense(num_classes, 
                                    activation='softmax',
                                    kernel_regularizer=regularizers.l2(0.001),
                                    kernel_initializer='he_normal')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'],
                  weighted_metrics=['accuracy'])  
    return model

# **Prepare TwoHigh Dataset**
num_classes = 52
random_state = 42
calibration_size = 0.1  
test_size = 0.2  
train_size = 1 - (calibration_size + test_size)  

combination_name = "TwoLow"
dataset = TwoLow

# **Prepare dataset**
X_combined = np.empty((dataset.shape[0] * dataset.shape[2], dataset.shape[1]))
labels = np.empty((dataset.shape[0] * dataset.shape[2]), dtype=int)  

for card in range(dataset.shape[2]):
    responses = dataset[:, :, card]
    start_idx = card * dataset.shape[0]
    end_idx = start_idx + dataset.shape[0]
    X_combined[start_idx:end_idx, :] = responses
    labels[start_idx:end_idx] = card  

# **Split Data**
X_train, X_temp, y_train, y_temp = train_test_split(X_combined, labels, test_size=(test_size + calibration_size), stratify=labels, random_state=random_state)
X_test, X_calib, y_test, y_calib = train_test_split(X_temp, y_temp, test_size=(calibration_size / (test_size + calibration_size)), stratify=y_temp, random_state=random_state)

#  **Save Split Data**
os.makedirs('./Data_Splits', exist_ok=True)
np.save(f'./Data_Splits/x_train_{combination_name}.npy', X_train)
np.save(f'./Data_Splits/y_train_{combination_name}.npy', y_train)
np.save(f'./Data_Splits/x_test_{combination_name}.npy', X_test)
np.save(f'./Data_Splits/y_test_{combination_name}.npy', y_test)
np.save(f'./Data_Splits/x_calib_{combination_name}.npy', X_calib)
np.save(f'./Data_Splits/y_calib_{combination_name}.npy', y_calib)

# **Set up Checkpoints & Early Stopping**
checkpoint_dir = "./tf_ckpts_TwoLow"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.weights.h5")  # Fixed the format

#  Save the best model checkpoint based on validation accuracy
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,  
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

#  Stop training early if validation loss doesn't improve for 5 epochs
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# **Train the Model with GPU**
strategy = tf.distribute.MirroredStrategy()  # Enables multi-GPU training
with strategy.scope():
    model = KerasClassifier(
        model=build_model,
        input_shape=(X_train.shape[1], 1),
        num_classes=num_classes,
        dropout_rate=0.5,  
        epochs=100,  
        batch_size=32,
        verbose=1,
        callbacks=[checkpoint_callback, early_stopping_callback]  
    )

#  **Train the Model**
model.fit(X_train, y_train, validation_split=0.10)

# **Save the Best Model Separately**
best_model_path = "MD/TwoLow_best.h5"
os.makedirs("MD", exist_ok=True)
model.model_.save(best_model_path)
print(f" Best model saved as {best_model_path}")

#  **Wrap and Fit MAPIE Classifier**
print(" Fitting MAPIE classifier...")
mapie_clf = MapieClassifier(estimator=model, method="score", cv="prefit")
mapie_clf.fit(X_calib, y_calib)
print(" MAPIE classifier fitted successfully.")

In [4]:
####### 5. TRAINING (TWOTYPES) DATASET #####

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from mapie.classification import MapieClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasClassifier  #  SciKeras Wrapper for TensorFlow Model


#  **Set random seed for reproducibility**
tf.random.set_seed(42)
np.random.seed(42)

#  **Load and Normalize Individual Datasets**
data_one_high = np.load("OneHigh_.npy", allow_pickle=True)
data_one_low = np.load("OneLow_.npy", allow_pickle=True)

scaler = MinMaxScaler()
def normalize_data(data):
    for i in range(52):  # 52 NFC tag classes
        data_abs = np.abs(data[:, :, i])
        data[:, :, i] = scaler.fit_transform(data_abs)
    return data.astype(np.float32)  

OneHigh = normalize_data(data_one_high)
OneLow = normalize_data(data_one_low)

#  **Combine Datasets to Create `TwoTypes`**
TwoTypes = np.concatenate((OneHigh, OneLow), axis=1)

#  **Updated CNN-ResNet-LSTM Model Architecture**
def build_model(input_shape, num_classes, dropout_rate=0.5):
    input_layer = tf.keras.layers.Input(input_shape)

    # **Initial Convolutional Layer**
    x = tf.keras.layers.Conv1D(32, 
                               kernel_size=8, 
                               padding='same', 
                               activation='relu',
                               kernel_regularizer=regularizers.l2(0.001),
                               kernel_initializer='he_normal')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)

    # **Residual Block Function**
    def residual_block(x, filters, kernel_size=8):
        shortcut = x
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation='relu',
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation=None,
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = tf.keras.layers.Conv1D(filters, 
                                              kernel_size=1, 
                                              padding='same',
                                              kernel_regularizer=regularizers.l2(0.001),
                                              kernel_initializer='he_normal')(shortcut)
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    # **Residual Block and LSTM Layers**
    x = residual_block(x, 32, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # **LSTM Layer**
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(64, 
                             return_sequences=True,
                             kernel_initializer='glorot_uniform')(x)

    x = residual_block(x, 64, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # **Output Layer**
    outputs = tf.keras.layers.Dense(num_classes, 
                                    activation='softmax',
                                    kernel_regularizer=regularizers.l2(0.001),
                                    kernel_initializer='he_normal')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'],
                  weighted_metrics=['accuracy'])  
    return model

# **Prepare `TwoTypes` Dataset**
num_classes = 52
random_state = 42
calibration_size = 0.1  
test_size = 0.2  
train_size = 1 - (calibration_size + test_size)  

combination_name = "TwoTypes"
dataset = TwoTypes

#  **Prepare dataset**
X_combined = np.empty((dataset.shape[0] * dataset.shape[2], dataset.shape[1]), dtype=np.float32)
labels = np.empty((dataset.shape[0] * dataset.shape[2]), dtype=int)  

for card in range(dataset.shape[2]):
    responses = dataset[:, :, card]
    start_idx = card * dataset.shape[0]
    end_idx = start_idx + dataset.shape[0]
    X_combined[start_idx:end_idx, :] = responses
    labels[start_idx:end_idx] = card  

# **Split Data**
X_train, X_temp, y_train, y_temp = train_test_split(X_combined, labels, test_size=(test_size + calibration_size), stratify=labels, random_state=random_state)
X_test, X_calib, y_test, y_calib = train_test_split(X_temp, y_temp, test_size=(calibration_size / (test_size + calibration_size)), stratify=y_temp, random_state=random_state)

#  **Save Split Data**
os.makedirs('./Data_Splits', exist_ok=True)
np.save(f'./Data_Splits/x_train_{combination_name}.npy', X_train)
np.save(f'./Data_Splits/y_train_{combination_name}.npy', y_train)
np.save(f'./Data_Splits/x_test_{combination_name}.npy', X_test)
np.save(f'./Data_Splits/y_test_{combination_name}.npy', y_test)
np.save(f'./Data_Splits/x_calib_{combination_name}.npy', X_calib)
np.save(f'./Data_Splits/y_calib_{combination_name}.npy', y_calib)

#  **Set up Checkpoints & Early Stopping**
checkpoint_dir = "./tf_ckpts_TwoTypes"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.weights.h5")  

# Save the best model checkpoint based on validation accuracy
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

#  Stop training early if validation loss doesn't improve for 5 epochs
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

#  **Train the Model**
model = KerasClassifier(
    model=build_model,
    input_shape=(X_train.shape[1], 1),
    num_classes=num_classes,
    dropout_rate=0.5,  
    epochs=100,  
    batch_size=32,
    verbose=1,
    callbacks=[checkpoint_callback, early_stopping_callback]  
)

# **Train the Model**
model.fit(X_train, y_train, validation_split=0.10)

#  **Save the Best Model Separately**
best_model_path = "MD/TwoTypes_best.h5"
os.makedirs("MD", exist_ok=True)
model.model_.save(best_model_path)
print(f" Best model saved as {best_model_path}")

#  **Wrap and Fit MAPIE Classifier**
print(" Fitting MAPIE classifier...")
mapie_clf = MapieClassifier(estimator=model, method="score", cv="prefit")
mapie_clf.fit(X_calib, y_calib)
print(" MAPIE classifier fitted successfully.")

In [5]:
####### 6. TRAINING (THREETYPES) DATASET #####

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from mapie.classification import MapieClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasClassifier  #  SciKeras Wrapper for TensorFlow Model

# **Set random seed for reproducibility**
tf.random.set_seed(42)
np.random.seed(42)

#  **Load and Normalize Individual Datasets**
data_one_high = np.load("OneHigh_.npy", allow_pickle=True)
data_one_low = np.load("OneLow_.npy", allow_pickle=True)
data_two_high = np.load("TwoHigh_.npy", allow_pickle=True)

scaler = MinMaxScaler()
def normalize_data(data):
    for i in range(52):  # 52 NFC tag classes
        data_abs = np.abs(data[:, :, i])
        data[:, :, i] = scaler.fit_transform(data_abs)
    return data.astype(np.float32)  # Ensure data is float32

OneHigh = normalize_data(data_one_high)
OneLow = normalize_data(data_one_low)
TwoHigh = normalize_data(data_two_high)

#  **Combine Datasets to Create `ThreeTypes`**
ThreeTypes = np.concatenate((OneHigh, OneLow, TwoHigh), axis=1)

#  **Updated CNN-ResNet-LSTM Model Architecture**
def build_model(input_shape, num_classes, dropout_rate=0.5):
    input_layer = tf.keras.layers.Input(input_shape)

    # **Initial Convolutional Layer**
    x = tf.keras.layers.Conv1D(32, 
                               kernel_size=8, 
                               padding='same', 
                               activation='relu',
                               kernel_regularizer=regularizers.l2(0.001),
                               kernel_initializer='he_normal')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)

    # **Residual Block Function**
    def residual_block(x, filters, kernel_size=8):
        shortcut = x
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation='relu',
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation=None,
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = tf.keras.layers.Conv1D(filters, 
                                              kernel_size=1, 
                                              padding='same',
                                              kernel_regularizer=regularizers.l2(0.001),
                                              kernel_initializer='he_normal')(shortcut)
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    # **Residual Block and LSTM Layers**
    x = residual_block(x, 32, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # **LSTM Layer**
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(64, 
                             return_sequences=True,
                             kernel_initializer='glorot_uniform')(x)

    x = residual_block(x, 64, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # **Output Layer**
    outputs = tf.keras.layers.Dense(num_classes, 
                                    activation='softmax',
                                    kernel_regularizer=regularizers.l2(0.001),
                                    kernel_initializer='he_normal')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'],
                  weighted_metrics=['accuracy'])  
    return model

# **Prepare `ThreeTypes` Dataset**
num_classes = 52
random_state = 42
calibration_size = 0.1  
test_size = 0.2  
train_size = 1 - (calibration_size + test_size)  

combination_name = "ThreeTypes"
dataset = ThreeTypes

#  **Prepare dataset**
X_combined = np.empty((dataset.shape[0] * dataset.shape[2], dataset.shape[1]), dtype=np.float32)
labels = np.empty((dataset.shape[0] * dataset.shape[2]), dtype=int)  

for card in range(dataset.shape[2]):
    responses = dataset[:, :, card]
    start_idx = card * dataset.shape[0]
    end_idx = start_idx + dataset.shape[0]
    X_combined[start_idx:end_idx, :] = responses
    labels[start_idx:end_idx] = card  

#  **Split Data**
X_train, X_temp, y_train, y_temp = train_test_split(X_combined, labels, test_size=(test_size + calibration_size), stratify=labels, random_state=random_state)
X_test, X_calib, y_test, y_calib = train_test_split(X_temp, y_temp, test_size=(calibration_size / (test_size + calibration_size)), stratify=y_temp, random_state=random_state)

#  **Save Split Data**
os.makedirs('./Data_Splits', exist_ok=True)
np.save(f'./Data_Splits/x_train_{combination_name}.npy', X_train)
np.save(f'./Data_Splits/y_train_{combination_name}.npy', y_train)
np.save(f'./Data_Splits/x_test_{combination_name}.npy', X_test)
np.save(f'./Data_Splits/y_test_{combination_name}.npy', y_test)
np.save(f'./Data_Splits/x_calib_{combination_name}.npy', X_calib)
np.save(f'./Data_Splits/y_calib_{combination_name}.npy', y_calib)

#  **Set up Checkpoints & Early Stopping**
checkpoint_dir = "./tf_ckpts_ThreeTypes"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.weights.h5")  

#  Save the best model checkpoint based on validation accuracy
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Stop training early if validation loss doesn't improve for 5 epochs
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

#  **Train the Model**
model = KerasClassifier(
    model=build_model,
    input_shape=(X_train.shape[1], 1),
    num_classes=num_classes,
    dropout_rate=0.5,  
    epochs=100,  
    batch_size=32,
    verbose=1,
    callbacks=[checkpoint_callback, early_stopping_callback]  
)

# **Train the Model**
model.fit(X_train, y_train, validation_split=0.10)

#  **Save the Best Model Separately**
best_model_path = "MD/ThreeTypes_best.h5"
os.makedirs("MD", exist_ok=True)
model.model_.save(best_model_path)
print(f" Best model saved as {best_model_path}")

#  **Wrap and Fit MAPIE Classifier**
print(" Fitting MAPIE classifier...")
mapie_clf = MapieClassifier(estimator=model, method="score", cv="prefit")
mapie_clf.fit(X_calib, y_calib)
print(" MAPIE classifier fitted successfully.")

In [6]:
####### 7. TRAINING (FOURTYPES) DATASET #####

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from mapie.classification import MapieClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasClassifier  #  SciKeras Wrapper for TensorFlow Model

# **Set random seed for reproducibility**
tf.random.set_seed(42)
np.random.seed(42)

#  **Load and Normalize Individual Datasets**
data_one_high = np.load("OneHigh_.npy", allow_pickle=True)
data_one_low = np.load("OneLow_.npy", allow_pickle=True)
data_two_high = np.load("TwoHigh_.npy", allow_pickle=True)
data_two_low = np.load("TwoLow_.npy", allow_pickle=True)

scaler = MinMaxScaler()
def normalize_data(data):
    for i in range(52):  # 52 NFC tag classes
        data_abs = np.abs(data[:, :, i])
        data[:, :, i] = scaler.fit_transform(data_abs)
    return data.astype(np.float32)  

OneHigh = normalize_data(data_one_high)
OneLow = normalize_data(data_one_low)
TwoHigh = normalize_data(data_two_high)
TwoLow = normalize_data(data_two_low)

#  **Combine Datasets to Create `FourTypes`**
FourTypes = np.concatenate((OneHigh, OneLow, TwoHigh, TwoLow), axis=1)

#  **Updated CNN-ResNet-LSTM Model Architecture**
def build_model(input_shape, num_classes, dropout_rate=0.5):
    input_layer = tf.keras.layers.Input(input_shape)

    # **Initial Convolutional Layer**
    x = tf.keras.layers.Conv1D(32, 
                               kernel_size=8, 
                               padding='same', 
                               activation='relu',
                               kernel_regularizer=regularizers.l2(0.001),
                               kernel_initializer='he_normal')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)

    # **Residual Block Function**
    def residual_block(x, filters, kernel_size=8):
        shortcut = x
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation='relu',
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv1D(filters, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   activation=None,
                                   kernel_regularizer=regularizers.l2(0.001),
                                   kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = tf.keras.layers.Conv1D(filters, 
                                              kernel_size=1, 
                                              padding='same',
                                              kernel_regularizer=regularizers.l2(0.001),
                                              kernel_initializer='he_normal')(shortcut)
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    # **Residual Block and LSTM Layers**
    x = residual_block(x, 32, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # **LSTM Layer**
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(64, 
                             return_sequences=True,
                             kernel_initializer='glorot_uniform')(x)

    x = residual_block(x, 64, kernel_size=8)
    x = tf.keras.layers.MaxPooling1D(2)(x)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # **Output Layer**
    outputs = tf.keras.layers.Dense(num_classes, 
                                    activation='softmax',
                                    kernel_regularizer=regularizers.l2(0.001),
                                    kernel_initializer='he_normal')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'],
                  weighted_metrics=['accuracy'])  
    return model

#  **Prepare `FourTypes` Dataset**
num_classes = 52
random_state = 42
calibration_size = 0.1  
test_size = 0.2  
train_size = 1 - (calibration_size + test_size)  

combination_name = "FourTypes"
dataset = FourTypes

#  **Prepare dataset**
X_combined = np.empty((dataset.shape[0] * dataset.shape[2], dataset.shape[1]), dtype=np.float32)
labels = np.empty((dataset.shape[0] * dataset.shape[2]), dtype=int)  

for card in range(dataset.shape[2]):
    responses = dataset[:, :, card]
    start_idx = card * dataset.shape[0]
    end_idx = start_idx + dataset.shape[0]
    X_combined[start_idx:end_idx, :] = responses
    labels[start_idx:end_idx] = card  

#  **Split Data**
X_train, X_temp, y_train, y_temp = train_test_split(X_combined, labels, test_size=(test_size + calibration_size), stratify=labels, random_state=random_state)
X_test, X_calib, y_test, y_calib = train_test_split(X_temp, y_temp, test_size=(calibration_size / (test_size + calibration_size)), stratify=y_temp, random_state=random_state)

#  **Save Split Data**
os.makedirs('./Data_Splits', exist_ok=True)
np.save(f'./Data_Splits/x_train_{combination_name}.npy', X_train)
np.save(f'./Data_Splits/y_train_{combination_name}.npy', y_train)
np.save(f'./Data_Splits/x_test_{combination_name}.npy', X_test)
np.save(f'./Data_Splits/y_test_{combination_name}.npy', y_test)
np.save(f'./Data_Splits/x_calib_{combination_name}.npy', X_calib)
np.save(f'./Data_Splits/y_calib_{combination_name}.npy', y_calib)

#  **Set up Checkpoints & Early Stopping**
checkpoint_dir = "./tf_ckpts_FourTypes"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.weights.h5")  

#  Save the best model checkpoint based on validation accuracy
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Stop training early if validation loss doesn't improve for 5 epochs
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

#  **Train the Model**
model = KerasClassifier(
    model=build_model,
    input_shape=(X_train.shape[1], 1),
    num_classes=num_classes,
    dropout_rate=0.5,  
    epochs=100,  
    batch_size=32,
    verbose=1,
    callbacks=[checkpoint_callback, early_stopping_callback]  
)

#  **Train the Model**
model.fit(X_train, y_train, validation_split=0.10)

# **Save the Best Model Separately**
best_model_path = "MD/FourTypes_best.h5"
os.makedirs("MD", exist_ok=True)
model.model_.save(best_model_path)
print(f" Best model saved as {best_model_path}")

# **Wrap and Fit MAPIE Classifier**
print("Fitting MAPIE classifier...")
mapie_clf = MapieClassifier(estimator=model, method="score", cv="prefit")
mapie_clf.fit(X_calib, y_calib)
print(" MAPIE classifier fitted successfully.")